In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import fastf1 as ff1

In [69]:
sessionQuali = ff1.get_session(2024, "Baku", 'Q')
sessionQuali.load(laps=True, telemetry=True, weather=True)

core           INFO 	Loading data for Azerbaijan Grand Prix - Qualifying [v3.4.4]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            IN

In [70]:
laps = sessionQuali.laps.pick_team("Ferrari")

C:\Users\benot\AppData\Roaming\Python\Python311\site-packages\fastf1\core.py:3073: FutureWarning: pick_team is deprecated and will be removed in a future release. Use pick_teams instead.
  warnings.warn(("pick_team is deprecated and will be removed"


In [71]:
quick_laps = laps.pick_quicklaps()

In [64]:
quick_laps.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate'],
      dtype='object')

In [72]:
q1, q2, q3 = quick_laps.split_qualifying_sessions()
q1[["LapTime","DriverNumber","Compound","TyreLife"]], q2[["LapTime","DriverNumber","Compound","TyreLife"]], q3[["LapTime","DriverNumber","Compound","TyreLife"]]

(                  LapTime DriverNumber Compound  TyreLife
 1  0 days 00:01:43.595000           16     SOFT       2.0
 4  0 days 00:01:42.775000           16     SOFT       5.0
 41 0 days 00:01:43.517000           55     SOFT       3.0
 46 0 days 00:01:43.357000           55     SOFT       8.0,
                   LapTime DriverNumber Compound  TyreLife
 7  0 days 00:01:43.422000           16     SOFT       8.0
 10 0 days 00:01:42.056000           16     SOFT       2.0
 49 0 days 00:01:43.225000           55     SOFT      11.0
 52 0 days 00:01:42.503000           55     SOFT       2.0,
                   LapTime DriverNumber Compound  TyreLife
 15 0 days 00:01:41.610000           16     SOFT       2.0
 18 0 days 00:01:41.365000           16     SOFT       2.0
 57 0 days 00:01:41.815000           55     SOFT       2.0
 60 0 days 00:01:41.805000           55     SOFT       2.0)

In [51]:
telemetry_frames = []
for _,lap in quick_laps.iterrows():
        telemetry = lap.get_telemetry()
        telemetry['Distance'] = telemetry['Distance'].clip(lower=0)  # Set negative distance values to 0
        # telemetry['LapNumber'] = lap['LapNumber']
        # telemetry['DriverNumber'] = driver_number
        telemetry['Year'] = "2024"
        telemetry['Event'] = "Italian Gran Prix"
        telemetry["Team"] = "Ferrari"
        telemetry_frames.append(telemetry)

    # Combine telemetry frames if available
tel = pd.concat(telemetry_frames) if telemetry_frames else None

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core      

In [52]:
#check if missing values
tel.isnull().sum()

Date                        0
SessionTime                 0
DriverAhead                 0
DistanceToDriverAhead    1228
Time                        0
RPM                         0
Speed                       0
nGear                       0
Throttle                    0
Brake                       0
DRS                         0
Source                      0
Distance                    0
RelativeDistance            0
Status                      0
X                           0
Y                           0
Z                           0
Year                        0
Event                       0
Team                        0
dtype: int64

In [61]:
#pick all the teams name of the session
sessionQuali.laps["Team"].unique()

array(['McLaren', 'Mercedes', 'Ferrari', 'Red Bull Racing', 'Williams',
       'Haas F1 Team', 'Aston Martin', 'RB', 'Alpine', 'Kick Sauber'],
      dtype=object)